In [70]:
import pandas as pd
import numpy as np
import string
from random import sample
import torch, torch.nn as nn

from transformers import BertForMaskedLM, BertTokenizer, pipeline, AutoTokenizer, AutoModelForMaskedLM

In [71]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))

cuda device is available


In [72]:
df = pd.read_excel('hackaton_result_dataset.xlsx')

>***Восстановление  слов в исходном тексте через маскирование*** 

In [30]:
model = AutoModelForMaskedLM.from_pretrained('ai-forever/ruRoberta-large')
tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruRoberta-large', do_lower_case=False)
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [74]:
# Генерация текста
def generate_sample(text):
    if type(text) == str:
        text = text.split()
        labels_for_mask = len(text)//2 + 1
        masked_labels = 0
        for i in range(len(text)):
            if len(text[i]) > 3 and masked_labels < labels_for_mask:
                text[i] = '<mask>'
                m = unmasker(' '.join(text))[2]['token_str']
                text[i] = m
                masked_labels += 1
        return ' '.join(text)
    else:
        return text

In [75]:
df['Generated text'] = np.nan

In [78]:
from tqdm.notebook import trange

for i in trange(0, len(df['Generated text'])):
    if len(df['model_annotation'][i].split()) > 2:
        df['Generated text'][i] = generate_sample(df['model_annotation'][i])
    else:
        df['Generated text'][i] = df['model_annotation'][i]


  0%|          | 0/6508 [00:00<?, ?it/s]

C:\Users\olg-k\AppData\Local\Temp\ipykernel_7276\1576650618.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Generated text'][i] = generate_sample(df['model_annotation'][i])
C:\Users\olg-k\AppData\Local\Temp\ipykernel_7276\1576650618.

>***Расчет косинусного расстояния между исходным пердложением и сгенерированным***

In [80]:
from sentence_transformers import SentenceTransformer, util
model_sent = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

c:\ProgramData\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [81]:
df['cosin_similarity'] = np.nan

In [89]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
for i in trange(0, len(df['Generated text'])):
    embedding1 = model_sent.encode(df['Generated text'][i], convert_to_tensor=True)
    embedding2 = model_sent.encode(df['model_annotation'][i], convert_to_tensor=True)

    df['cosin_similarity'][i] = cos(embedding1, embedding2)


  0%|          | 0/6508 [00:00<?, ?it/s]

In [96]:
df['cosin_similarity'] = df['cosin_similarity'].apply(lambda x: float(x.cpu().numpy()))

In [97]:
df[40:70]

,model_annotation,human_markup,audio_path,label,Generated text,cosin_similarity
40,я спросил у меня нет,ты спросил у меня нет,s3://ap-training-set/xacaton_openstt/part_1/05...,0,я думаю у меня нет,0.608466
41,номер нет этот дом,номер нет это дом,s3://ap-training-set/xacaton_openstt/part_1/05...,0,Да нет же дом,0.833012
42,про военной сбор пел,про военные сборы пел,s3://ap-training-set/xacaton_openstt/part_1/05...,1,про общий . пел,0.583448
43,что такой,ты кто такой,s3://ap-training-set/xacaton_openstt/part_1/05...,1,что такой,1.000000
44,ты мне чего звонишь,ты мне чё звонишь,s3://ap-training-set/xacaton_openstt/part_1/05...,0,ты мне постоянно !,0.688030
45,туда подлей тебя и нету на свете даже,ну да подлей тебя и нету на свете даже,s3://ap-training-set/xacaton_openstt/part_1/05...,0,ну может ее и не на фото даже,0.642406
46,любой допреза ваши деньги,любой каприз за ваши деньги,s3://ap-training-set/xacaton_openstt/part_1/05...,0,Я взял бы деньги,0.703865
47,ну вы дежурные что я должна сделать,ну вы дежурный и что я должна сделать,s3://ap-training-set/xacaton_openstt/part_1/05...,0,ну вы поняли что я хотела ),0.710855
48,да это страшно ты же мне что то там э ну тоже ...,тааак это страшно ты же мне шо то там эээ ну т...,s3://ap-training-set/xacaton_openstt/part_1/05...,0,да это ! ты же мне что то там э ну то говори...,0.856504
49,какому времени,какому времени,s3://ap-training-set/xacaton_openstt/part_1/05...,0,какому времени,1.000000


In [98]:
df['cosin_similarity'].corr(df['label'])

0.09693389366715503

In [99]:
df.to_excel("result.xlsx")